In [ ]:
import pandas as pd 
import numpy as np 
import geopandas as gpd
import rasterio
import pystac_client

import sys


In [4]:

def extract_gretter_fire_per_year(year=2022, tile='22LHK'): 
    service = pystac_client.Client.open('https://data.inpe.br/bdc/stac/v1/')
    im= pd.read_json(f'early_burned_area_{year}_{tile}_30.json')
    after=im[im['ba_detect'] == 1].sort_values(by='firespots_that', ascending=False).iloc[0]['item_after']
    day1 = im.iloc[0]['day_before']
    day2 = im.iloc[0]['day_after']
    before=im[im['ba_detect'] == 1].sort_values(by='firespots_that', ascending=False).iloc[0]['item_before']
    item_search_before = service.search(ids=[f'{before}'],
                                collections=['S2_L2A-1'],
                                )
    item_search_before.matched()
    item_search_after = service.search(ids=[f'{after}'],
                                collections=['S2_L2A-1'],
                                )
    item_search_after.matched()

    for item in item_search_before.items():
        assets_before = item.assets
    assets_before

    for item in item_search_after.items():
        assets_after = item.assets
    assets_after

    nir_asset_after = assets_after['B8A']
    nir_asset_after.href
    swir_asset_after = assets_after['B12']
    swir_asset_after.href
    swir1_asset_after = assets_after['B11']
    swir1_asset_after.href
    scl_asset_after = assets_after['SCL']
    scl_asset_after.href
    nir_asset_before = assets_before['B8A']
    nir_asset_after.href
    swir_asset_before = assets_before['B12']
    swir_asset_after.href
    swir1_asset_before = assets_before['B11']
    swir1_asset_before.href
    scl_asset_before = assets_after['SCL']
    scl_asset_before.href

    generating_after_and_before(swir1_asset_after,swir_asset_after,swir1_asset_before,swir_asset_before,year,tile,'nbr_swir')
    generating_after_and_before(swir1_asset_after,nir_asset_after,swir1_asset_before,nir_asset_before,year,tile,'nbr')
    dnbr_dnbrswir(year,tile)

    create_rgb_image(year, tile,'nbr')
    create_rgb_image(year, tile,'nbr_swir')
    print(f"NBR salvo em: after.tif")

def generating_after_and_before(swir1_asset_after,swir_asset_after,swir1_asset_before,swir_asset_before,year,tile,tipo):
        # Abrindo os rasters
    with rasterio.open(swir1_asset_after.href) as src_b11, rasterio.open(swir_asset_after.href) as src_b12:
        # Lendo as bandas como arrays
        b11 = src_b11.read(1).astype('float32')
        b12 = src_b12.read(1).astype('float32')

        # Evitando divisão por zero
        soma = b11 + b12
        soma[soma == 0] = np.nan  # Define como NaN onde a soma for zero

        # Calculando o NBR
        nbr = (b11 - b12) / soma

        # Configuração dos metadados para o novo raster
        meta = src_b11.meta.copy()
        meta.update(dtype=rasterio.float32, count=1)
        # Salvando o raster do NBR
        with rasterio.open(f'{tipo}_after_{year}_{tile}.tif', 'w', **meta) as dst:
            dst.write(nbr, 1)

    with rasterio.open(swir1_asset_before.href) as src_b11, rasterio.open(swir_asset_before.href) as src_b12:
        # Lendo as bandas como arrays
        b11 = src_b11.read(1).astype('float32')
        b12 = src_b12.read(1).astype('float32')
        
        # Evitando divisão por zero
        soma = b11 + b12
        soma[soma == 0] = np.nan  # Define como NaN onde a soma for zero

        # Calculando o NBR
        nbr = (b11 - b12) / soma

        # Configuração dos metadados para o novo raster
        meta = src_b11.meta.copy()
        meta.update(dtype=rasterio.float32, count=1)

        # Salvando o raster do NBR
        with rasterio.open(f'{tipo}_before_{year}_{tile}.tif', 'w', **meta) as dst:
            dst.write(nbr, 1)

def create_rgb_image(year=2022, tile='22LHK',tipo='nbr'):
    # Abrindo os rasters antes e depois do incêndio
    with rasterio.open(f'{tipo}_before_{year}_{tile}.tif') as src_b, \
         rasterio.open(f'{tipo}_after_{year}_{tile}.tif') as src_r, \
         rasterio.open(f'd_{tipo}_{year}_{tile}.tif') as src_g:

        # Lendo os dados das imagens
        red = src_r.read(1).astype(np.float32)  # before
        green = src_g.read(1).astype(np.float32)  # before_fire
        blue = src_b.read(1).astype(np.float32)  # after_fire

        # Criando um array empilhado com os três canais
        rgb = np.stack([red, green, blue], axis=0)

        # Configuração dos metadados para a nova imagem
        meta = src_r.meta.copy()
        meta.update(dtype=rasterio.float32, count=3)

        # Salvando a imagem RGB
        with rasterio.open(f'rgb_{tipo}_{year}_{tile}.tif', 'w', **meta) as dst:
            dst.write(rgb)

    print(f"Imagem RGB salva em: rgb_fire_{year}_{tile}.tif")

def dnbr_dnbrswir(year,tile):
    with rasterio.open(f'nbr_before_{year}_{tile}.tif') as sr1c, \
        rasterio.open(f'nbr_after_{year}_{tile}.tif') as sr2c, \
        rasterio.open(f'nbr_swir_before_{year}_{tile}.tif') as sr3c, \
        rasterio.open(f'nbr_swir_after_{year}_{tile}.tif') as sr4c:
        nbr_calculos(sr1c,sr2c,'nbr',year,tile)
        nbr_calculos(sr3c,sr4c,'nbr_swir',year,tile)
        # Ler os dados das bandas

def nbr_calculos(sr1c,sr2c,tipo,year,tile):
        before = sr1c.read(1).astype(np.float32)
        after = sr2c.read(1).astype(np.float32)
        before = sr1c.read(1).astype(np.float32)
        after = sr2c.read(1).astype(np.float32)
        
        # Calcular a diferença e aplicar sqrt em valores absolutos
        dmask = np.sqrt(np.abs(before - after))
        
        # Copiar metadados e atualizar tipo de dado
        meta = sr1c.meta.copy()
        meta.update(dtype=rasterio.float32, count=1)
        
        # Salvar o resultado
        with rasterio.open(f'd_{tipo}_{year}_{tile}.tif', 'w', **meta) as dst:
            dst.write(dmask[np.newaxis, :, :])  # Adiciona dimensão da banda

if __name__=='__main__':
    year = 2022
    print(year)
    extract_gretter_fire_per_year(year,'23KRT')
    extract_gretter_fire_per_year(year,'24KTC')
    year = 2024
    print(year)
    extract_gretter_fire_per_year(year,'23KRT')
    extract_gretter_fire_per_year(year,'24KTC')
    

2022
Imagem RGB salva em: rgb_fire_2022_23KRT.tif
Imagem RGB salva em: rgb_fire_2022_23KRT.tif
NBR salvo em: after.tif
Imagem RGB salva em: rgb_fire_2022_24KTC.tif
Imagem RGB salva em: rgb_fire_2022_24KTC.tif
NBR salvo em: after.tif
2024
Imagem RGB salva em: rgb_fire_2024_23KRT.tif
Imagem RGB salva em: rgb_fire_2024_23KRT.tif
NBR salvo em: after.tif
Imagem RGB salva em: rgb_fire_2024_24KTC.tif
Imagem RGB salva em: rgb_fire_2024_24KTC.tif
NBR salvo em: after.tif


In [ ]:
for year in range(3):
    year = 2022 + year
    extract_gretter_fire_per_year(year,'22LHK')
    extract_gretter_fire_per_year(year,'22LHL')
    extract_gretter_fire_per_year(year,'23LKE')
    extract_gretter_fire_per_year(year,'23LKF')